# OPHI vs. ML Baseline for PM2.5 Prediction
This notebook compares a symbolic OPHI model to a standard Random Forest regressor on simulated PM2.5 data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

# Simulate PM2.5 dataset
np.random.seed(42)
days = pd.date_range(end=datetime.today(), periods=90)
pm25 = np.clip(np.random.normal(loc=70, scale=25, size=len(days)), 10, 200)
temperature = np.random.uniform(10, 35, size=len(days))
humidity = np.random.uniform(30, 90, size=len(days))
wind_speed = np.random.uniform(1, 10, size=len(days))

df = pd.DataFrame({
    "date": days,
    "pm25": pm25,
    "temperature": temperature,
    "humidity": humidity,
    "wind_speed": wind_speed
})

df["pm25_prev"] = df["pm25"].shift(1)
df = df.dropna().reset_index(drop=True)

In [ ]:
# Train/test split
train = df.iloc[:-20]
test = df.iloc[-20:]

# Random Forest baseline
features = ["pm25_prev", "temperature", "humidity", "wind_speed"]
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(train[features], train["pm25"])
test["rf_pred"] = rf.predict(test[features])

In [ ]:
# OPHI model
def ophi_predict(state, bias, alpha):
    return (state + bias) * alpha

test["bias"] = np.clip((10 - test["wind_speed"]) * 0.5, 0.1, 5.0)
test["alpha"] = np.clip(1 + (test["temperature"] - 15) / 50, 1.0, 1.5)
test["ophi_pred"] = ophi_predict(test["pm25_prev"], test["bias"], test["alpha"])

In [ ]:
# Evaluate models
print("Random Forest RMSE:", mean_squared_error(test["pm25"], test["rf_pred"], squared=False))
print("Random Forest R²:", r2_score(test["pm25"], test["rf_pred"]))
print("Improved OPHI RMSE:", mean_squared_error(test["pm25"], test["ophi_pred"], squared=False))
print("Improved OPHI R²:", r2_score(test["pm25"], test["ophi_pred"]))

In [ ]:
# Plot comparison
plt.figure(figsize=(12, 5))
plt.plot(test["date"], test["pm25"], label="Actual PM2.5", marker='o')
plt.plot(test["date"], test["ophi_pred"], label="Improved OPHI Ω", linestyle='--', marker='x')
plt.xticks(rotation=45)
plt.title("Improved OPHI Ω vs Actual PM2.5 Over Time")
plt.xlabel("Date")
plt.ylabel("PM2.5 (µg/m³)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()